In [19]:
## ML EDA
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df= pd.read_parquet("../artifacts/cleandata.parquet")
df.head()

,fit,user_id,item_id,weight_lbs,rating,rented_for,body_type,category,size,age,review_date,bust_size_num,bust_size_letter,height_in,review_month,review_day_of_month,review_year,recommend
0,fit,420272,2260466,137.0,10.0,vacation,hourglass,romper,14,28.0,2016-04-20,34.0,d,68.0,4,20,2016,1
1,fit,273551,153475,132.0,10.0,other,straight & narrow,gown,12,36.0,2013-06-18,34.0,b,66.0,6,18,2013,1
2,fit,360448,1063761,NaN,10.0,party,NaN,sheath,4,116.0,2015-12-14,NaN,NaN,64.0,12,14,2015,1
3,fit,909926,126335,135.0,8.0,formal affair,pear,dress,8,34.0,2014-02-12,34.0,c,65.0,2,12,2014,0
4,fit,151944,616682,145.0,10.0,wedding,athletic,gown,12,27.0,2016-09-26,34.0,b,69.0,9,26,2016,1


In [49]:
# how many times did users rate multiple times?
count_purchases = df["user_id"].value_counts().sort_values(ascending=True)
print(
    f"Total number of users are {df.user_id.nunique()}. Things were rented multiple times by {count_purchases[count_purchases > 1].shape[0]} (unique) users. The max number of rentals by a single user was {count_purchases[count_purchases > 1].max()} and on average users rented {count_purchases[count_purchases > 1].mean():.2f} items."
)


Total number of users are 105571. Things were rented multiple times by 33747 (unique) users. The max number of rentals by a single user was 436 and on average users rented 3.58 items.


In [21]:
# find 99.9% percentile of # ratings per user
df["user_id"].value_counts().sort_values(ascending=True).quantile(
    0.999
) ##consider anything above this outlier?

30.430000000007567

In [27]:
# check how many times each item was rated more than once
count_ratings= df["item_id"].value_counts().sort_values(ascending=True)
print(f"There are {df['item_id'].nunique()} unique items out of which {count_ratings[count_ratings > 1].shape[0]} have been rated more than once. ")

There are 5850 unique items out of which 5509 have been rated more than once. 


In [41]:
#whats common with the most popular rated items?
most_popular_rated= count_ratings[count_ratings > 1].sort_values(ascending=False)[:1000]
most_popular_items=most_popular_rated.index
most_popular_df= df.copy()
most_popular_df['KEEP']= most_popular_df.item_id.isin(most_popular_items)
most_popular_df=most_popular_df[most_popular_df.KEEP==True]
cols= ['item_id','fit','rented_for','category']
most_popular_df_items=most_popular_df.loc[:,cols] #keeping only things that describe the item
most_popular_df_items.head()

,item_id,fit,rented_for,category
1,153475,fit,other,gown
2,1063761,fit,party,sheath
3,126335,fit,formal affair,dress
4,616682,fit,wedding,gown
7,130259,fit,formal affair,dress


In [45]:
most_popular_df_items.fit.value_counts(normalize=True) #not right

fit      0.751528
small    0.135418
large    0.113054
Name: fit, dtype: float64

In [46]:
most_popular_df_items.rented_for.value_counts(normalize=True)

wedding          0.359902
formal affair    0.261486
party            0.191412
other            0.081313
work             0.037243
date             0.031163
everyday         0.022699
vacation         0.014783
Name: rented_for, dtype: float64

In [47]:
most_popular_df_items.category.value_counts(normalize=True)

dress         0.480604
gown          0.295800
sheath        0.114413
shift         0.029263
jumpsuit      0.022627
maxi          0.016277
mini          0.013511
romper        0.012605
jacket        0.002360
shirtdress    0.002209
skirt         0.001939
frock         0.001629
blazer        0.001550
top           0.001272
suit          0.000978
print         0.000819
legging       0.000517
blouse        0.000461
leggings      0.000405
coat          0.000389
down          0.000374
Name: category, dtype: float64

In [48]:
#missing values
df.isna().sum()

fit                        0
user_id                    0
item_id                    0
weight_lbs             29982
rating                    82
rented_for                10
body_type              14637
category                   0
size                       0
age                      960
review_date                0
bust_size_num          18411
bust_size_letter       18411
height_in                677
review_month               0
review_day_of_month        0
review_year                0
recommend                  0
dtype: int64

In [51]:
#reccomend - not reccomend split
df.recommend.value_counts(normalize=True)

## reccomendations should be conditional on events, category, so on...

1    0.646798
0    0.353202
Name: recommend, dtype: float64